In [ ]:
# Importation des bibliothèques 
import geopandas as gpd
import rasterstats
from my_function import calculate_class_percentages, apply_decision_rules, compute_confusion_matrix

In [ ]:
# Définition des cheminss d'accès 
my_folder = '/home/onyxia/work/Depot_Git/results/data'
sample_filename = os.path.join(my_folder, 'sample/Sample_BD_foret_T31TCJ_v2.tif')
image_filename = "path_carte_d'essence_pixel"

In [ ]:
# Chargement des échantillons 
polygons = gpd.read_file(sample_filename)

In [ ]:
# Calcul des pourcentages des classes dans les polygones
class_percentages = calculate_class_percentages(polygons, image_filename)

In [ ]:
# Ajout des classifications prédites selon les règles de décision
polygons["code_predit"] = apply_decision_rules(class_percentages)

In [ ]:
# Sauvegarde de shapefile mis à jour
shapefile_classified = "Sample_BD_foret_T31TCJ_classified.shp"
polygons.to_file(shapefile_classified)

In [ ]:
# Calcul de  la matrice de confusion
confusion_matrix = compute_confusion_matrix(polygons, label_col="code", prediction_col="code_predit")
print(confusion_matrix)

In [ ]:
# Affichage de la matrice de confusion et le rapport
print("Matrice de confusion :")
print(results["confusion_matrix"])
print("\nRapport de classification :")
print(results["classification_report"])